# Bayesian Probabilistic Tensor Factorization (BPTF)



**Published**: January 4, 2020

**Download**: This Jupyter notebook is at our GitHub repository. If you want to evaluate the code, please download the notebook from the [**transdim**](https://github.com/xinychen/transdim/blob/master/imputer/BPTF.ipynb) repository.

This notebook shows how to implement the Bayesian Probabilistic Tensor Factorization (BPTF), a fully Bayesian tensor factorization model, on some real-world data sets. For an in-depth discussion of BPTF, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Liang Xiong, Xi Chen, Tzu-kuo Huang, Jeff Schneider, and Jaime Carbonell (2010). <b>Temporal Collaborative Filtering with Bayesian Probabilistic Tensor Factorization</b>. SIAM Data Mining 2010 (SDM 2010). <a href="https://www.cs.cmu.edu/~jgc/publication/PublicationPDF/Temporal_Collaborative_Filtering_With_Bayesian_Probabilidtic_Tensor_Factorization.pdf" title="PDF"><b>[PDF]</b></a> <a href="http://www.cs.cmu.edu/~lxiong/bptf/bptf.html" title="Matlab code"><b>[Matlab code]</b></a>
</font>
</div>

In [1]:
import numpy as np
from numpy.linalg import inv as inv
from numpy.random import normal as normrnd
from scipy.linalg import khatri_rao as kr_prod
from scipy.stats import wishart
from scipy.stats import invwishart
from numpy.linalg import solve as solve
from numpy.linalg import cholesky as cholesky_lower
from scipy.linalg import cholesky as cholesky_upper
from scipy.linalg import solve_triangular as solve_ut
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
def mvnrnd_pre(mu, Lambda):
    src = normrnd(size = (mu.shape[0],))
    return solve_ut(cholesky_upper(Lambda, overwrite_a = True, check_finite = False), 
                    src, lower = False, check_finite = False, overwrite_b = True) + mu

def cov_mat(mat, mat_bar):
    mat = mat - mat_bar
    return mat.T @ mat

def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

In [3]:
def sample_factor_u(tau_sparse_tensor, tau_ind, U, V, X, beta0 = 1):
    """Sampling M-by-R factor matrix U and its hyperparameters (mu_u, Lambda_u)."""
    
    dim1, rank = U.shape
    U_bar = np.mean(U, axis = 0)
    temp = dim1 / (dim1 + beta0)
    var_mu_hyper = temp * U_bar
    var_U_hyper = inv(np.eye(rank) + cov_mat(U, U_bar) + temp * beta0 * np.outer(U_bar, U_bar))
    var_Lambda_hyper = wishart.rvs(df = dim1 + rank, scale = var_U_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim1 + beta0) * var_Lambda_hyper)

    var1 = kr_prod(X, V).T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ ten2mat(tau_ind, 0).T).reshape([rank, rank, dim1]) + var_Lambda_hyper[:, :, None]
    var4 = var1 @ ten2mat(tau_sparse_tensor, 0).T + (var_Lambda_hyper @ var_mu_hyper)[:, None]
    for i in range(dim1):
        U[i, :] = mvnrnd_pre(solve(var3[:, :, i], var4[:, i]), var3[:, :, i])
        
    return U

In [4]:
def sample_factor_v(tau_sparse_tensor, tau_ind, U, V, X, beta0 = 1):
    """Sampling N-by-R factor matrix V and its hyperparameters (mu_v, Lambda_v)."""
    
    dim2, rank = V.shape
    V_bar = np.mean(V, axis = 0)
    temp = dim2 / (dim2 + beta0)
    var_mu_hyper = temp * V_bar
    var_V_hyper = inv(np.eye(rank) + cov_mat(V, V_bar) + temp * beta0 * np.outer(V_bar, V_bar))
    var_Lambda_hyper = wishart.rvs(df = dim2 + rank, scale = var_V_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (dim2 + beta0) * var_Lambda_hyper)

    var1 = kr_prod(X, U).T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ ten2mat(tau_ind, 1).T).reshape([rank, rank, dim2]) + var_Lambda_hyper[:, :, None]
    var4 = var1 @ ten2mat(tau_sparse_tensor, 1).T + (var_Lambda_hyper @ var_mu_hyper)[:, None]
    for j in range(dim2):
        V[j, :] = mvnrnd_pre(solve(var3[:, :, j], var4[:, j]), var3[:, :, j])
        
    return V

In [5]:
def sample_factor_x(tau_sparse_tensor, tau_ind, U, V, X, beta0 = 1):
    """Sampling T-by-R factor matrix X and its hyperparameters."""
    
    dim3, rank = X.shape
    var_mu_hyper = X[0, :] / (beta0 + 1)
    dx = X[1 :, :] - X[: -1, :]
    var_V_hyper = inv(np.eye(rank) + dx.T @ dx + beta0 * np.outer(X[0, :], X[0, :]) / (beta0 + 1))
    var_Lambda_hyper = wishart.rvs(df = dim3 + rank, scale = var_V_hyper)
    var_mu_hyper = mvnrnd_pre(var_mu_hyper, (beta0 + 1) * var_Lambda_hyper)
    
    var1 = kr_prod(V, U).T
    var2 = kr_prod(var1, var1)
    var3 = (var2 @ ten2mat(tau_ind, 2).T).reshape([rank, rank, dim3])
    var4 = var1 @ ten2mat(tau_sparse_tensor, 2).T
    for t in range(dim3):
        if t == 0:
            X[t, :] = mvnrnd_pre((X[t + 1, :] + var_mu_hyper) / 2, var3[:, :, t] + 2 * var_Lambda_hyper)
        elif t == dim3 - 1:
            temp1 = var4[:, t] + var_Lambda_hyper @ X[t - 1, :]
            temp2 = var3[:, :, t] + var_Lambda_hyper
            X[t, :] = mvnrnd_pre(solve(temp2, temp1), temp2)
        else:
            temp1 = var4[:, t] + var_Lambda_hyper @ (X[t - 1, :] + X[t + 1, :])
            temp2 = var3[:, :, t] + 2 * var_Lambda_hyper
            X[t, :] = mvnrnd_pre(solve(temp2, temp1), temp2)
            
    return X

In [6]:
def sample_precision_tau(sparse_tensor, tensor_hat, ind):
    var_alpha = 1e-6 + 0.5 * np.sum(ind)
    var_beta = 1e-6 + 0.5 * np.sum(((sparse_tensor - tensor_hat) ** 2) * ind)
    return np.random.gamma(var_alpha, 1 / var_beta)

In [7]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

In [8]:
def BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter):
    """Bayesian Probabilistic Tensor Factorization, BPTF."""
    
    dim = np.array(sparse_tensor.shape)
    U = init["U"]
    V = init["V"]
    X = init["X"]
    if np.isnan(sparse_tensor).any() == False:
        ind = sparse_tensor != 0
        pos_obs = np.where(ind)
        pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    elif np.isnan(sparse_tensor).any() == True:
        pos_test = np.where((dense_tensor != 0) & (np.isnan(sparse_tensor)))
        ind = ~np.isnan(sparse_tensor)
        pos_obs = np.where(ind)
        sparse_tensor[np.isnan(sparse_tensor)] = 0
    dense_test = dense_tensor[pos_test]
    del dense_tensor
    show_iter = 200
    tau = 1
    temp_hat = np.zeros(len(pos_test[0]))
    tensor_hat_plus = np.zeros(dim)
    for it in range(burn_iter + gibbs_iter):
        tau_ind = tau * ind
        tau_sparse_tensor = tau * sparse_tensor
        U = sample_factor_u(tau_sparse_tensor, tau_ind, U, V, X)
        V = sample_factor_v(tau_sparse_tensor, tau_ind, U, V, X)
        X = sample_factor_x(tau_sparse_tensor, tau_ind, U, V, X)
        tensor_hat = np.einsum('is, js, ts -> ijt', U, V, X)
        temp_hat += tensor_hat[pos_test]
        tau = sample_precision_tau(sparse_tensor, tensor_hat, ind)
        if it + 1 > burn_iter:
            tensor_hat_plus += tensor_hat
        if (it + 1) % show_iter == 0 and it < burn_iter:
            temp_hat = temp_hat / show_iter
            print('Iter: {}'.format(it + 1))
            print('MAPE: {:.6}'.format(compute_mape(dense_test, temp_hat)))
            print('RMSE: {:.6}'.format(compute_rmse(dense_test, temp_hat)))
            temp_hat = np.zeros(len(pos_test[0]))
            print()
    tensor_hat = tensor_hat_plus / gibbs_iter
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_test, tensor_hat[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_test, tensor_hat[pos_test])))
    print()
    
    return tensor_hat

## Evaluation on New York Taxi Data

**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Random missing (RM)
- 40% missing rate


In [9]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
rm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/rm_tensor.mat')['rm_tensor']
missing_rate = 0.4

## Random missing (RM)
binary_tensor = np.round(rm_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [10]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.488178
RMSE: 4.77707

Iter: 400
MAPE: 0.489905
RMSE: 4.8486

Iter: 600
MAPE: 0.4887
RMSE: 4.83621

Iter: 800
MAPE: 0.488249
RMSE: 4.83004

Iter: 1000
MAPE: 0.488196
RMSE: 4.8268

Imputation MAPE: 0.488657
Imputation RMSE: 4.82266

Running time: 930 seconds


**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Random missing (RM)
- 60% missing rate


In [11]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
rm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/rm_tensor.mat')['rm_tensor']
missing_rate = 0.6

## Random missing (RM)
binary_tensor = np.round(rm_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [12]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.486112
RMSE: 4.91823

Iter: 400
MAPE: 0.49029
RMSE: 5.04225

Iter: 600
MAPE: 0.49136
RMSE: 5.03241

Iter: 800
MAPE: 0.491822
RMSE: 5.02832

Iter: 1000
MAPE: 0.492322
RMSE: 5.02735

Imputation MAPE: 0.492777
Imputation RMSE: 5.02246

Running time: 911 seconds


**Scenario setting**:

- Tensor size: $30\times 30\times 1464$ (origin, destination, time)
- Non-random missing (NM)
- 40% missing rate


In [13]:
import scipy.io

dense_tensor = scipy.io.loadmat('../datasets/NYC-data-set/tensor.mat')['tensor'].astype(np.float32)
nm_tensor = scipy.io.loadmat('../datasets/NYC-data-set/nm_tensor.mat')['nm_tensor']
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        for i3 in range(61):
            binary_tensor[i1, i2, i3 * 24 : (i3 + 1) * 24] = np.round(nm_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [14]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.483286
RMSE: 5.03192

Iter: 400
MAPE: 0.486799
RMSE: 5.36387

Iter: 600
MAPE: 0.484757
RMSE: 5.38105

Iter: 800
MAPE: 0.485221
RMSE: 5.36304

Iter: 1000
MAPE: 0.485226
RMSE: 5.35716

Imputation MAPE: 0.485402
Imputation RMSE: 5.35146

Running time: 918 seconds


## Evaluation on Pacific Surface Temperature Data

**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Random missing (RM)
- 40% missing rate


In [15]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.4

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [16]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0164988
RMSE: 0.690786

Iter: 400
MAPE: 0.0163371
RMSE: 0.669877

Iter: 600
MAPE: 0.0163816
RMSE: 0.673931

Iter: 800
MAPE: 0.016492
RMSE: 0.699589

Iter: 1000
MAPE: 0.0164883
RMSE: 0.691353

Imputation MAPE: 0.0165326
Imputation RMSE: 0.687077

Running time: 477 seconds


**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Random missing (RM)
- 60% missing rate


In [17]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], dense_tensor.shape[2])
missing_rate = 0.6

## Random missing (RM)
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [18]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0170249
RMSE: 0.701465

Iter: 400
MAPE: 0.0167618
RMSE: 0.683806

Iter: 600
MAPE: 0.0166442
RMSE: 0.690136

Iter: 800
MAPE: 0.0167566
RMSE: 0.704163

Iter: 1000
MAPE: 0.0167664
RMSE: 0.69708

Imputation MAPE: 0.016802
Imputation RMSE: 0.689297

Running time: 478 seconds


**Scenario setting**:

- Tensor size: $30\times 84\times 396$ (location x, location y, month)
- Non-random missing (NM)
- 40% missing rate


In [19]:
import numpy as np
np.random.seed(1000)

dense_tensor = np.load('../datasets/Temperature-data-set/tensor.npy').astype(np.float32)
pos = np.where(dense_tensor[:, 0, :] > 50)
dense_tensor[pos[0], :, pos[1]] = 0
random_tensor = np.random.rand(dense_tensor.shape[0], dense_tensor.shape[1], int(dense_tensor.shape[2] / 3))
missing_rate = 0.4

## Non-random missing (NM)
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        for i3 in range(int(dense_tensor.shape[2] / 3)):
            binary_tensor[i1, i2, i3 * 3 : (i3 + 1) * 3] = np.round(random_tensor[i1, i2, i3] + 0.5 - missing_rate)
sparse_tensor = dense_tensor.copy()
sparse_tensor[binary_tensor == 0] = np.nan
sparse_tensor[sparse_tensor == 0] = np.nan

**Model setting**:

- Low rank: 30
- The number of burn-in iterations: 1000
- The number of Gibbs iterations: 200

In [20]:
import time
start = time.time()
dim1, dim2, dim3 = sparse_tensor.shape
rank = 30
init = {"U": 0.1 * np.random.randn(dim1, rank), "V": 0.1 * np.random.randn(dim2, rank),
        "X": 0.1 * np.random.randn(dim3, rank)}
burn_iter = 1000
gibbs_iter = 200
tensor_hat = BPTF(dense_tensor, sparse_tensor, init, burn_iter, gibbs_iter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iter: 200
MAPE: 0.0165672
RMSE: 0.68826

Iter: 400
MAPE: 0.0164919
RMSE: 0.685095

Iter: 600
MAPE: 0.0164591
RMSE: 0.682162

Iter: 800
MAPE: 0.016412
RMSE: 0.663817

Iter: 1000
MAPE: 0.0164254
RMSE: 0.6714

Imputation MAPE: 0.0164897
Imputation RMSE: 0.699385

Running time: 472 seconds


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>